In [1]:
from deep_translator import GoogleTranslator
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
from process_text import cleanup_sentence

In [3]:
import os
import ssl
ssl._create_unverified_context = ssl._create_unverified_context
os.environ['http_proxy'] = "http://proxy.statestr.com:80" 
os.environ['https_proxy'] = "http://proxy.statestr.com:80" 
os.environ['CURL_CA_BUNDLE'] = ""

In [4]:
def translate(text, source_lang, dest_lang):
    return GoogleTranslator(source=source_lang, target=dest_lang).translate(text)

In [7]:
EDIT_DISTANCE_METHOD = False

In [8]:
if EDIT_DISTANCE_METHOD:
    train_df_with_score = pd.read_csv('top_50_sections_scores.csv')
else:
    train_df_with_score = pd.read_csv('top_50_sections_scores_original.csv')
train_df_with_score

,section_name,score
0,Chairman’s Statement,258.323541
1,Chairman’s statement,248.386919
2,Highlights,204.876537
3,Chief Executive’s Review,117.712061
4,Chief Executive’s review,104.321819
5,start,93.945820
6,Financial Highlights,59.091292
7,Financial highlights,44.450977
8,Chief Executive’s Statement,38.947681
9,Chief Executive’s Report,32.266886


In [9]:
train_df_with_score['clean_english'] = train_df_with_score.section_name.apply(lambda x: cleanup_sentence(x))
train_df_with_score.head()

,section_name,score,clean_english
0,Chairman’s Statement,258.323541,chairman statement
1,Chairman’s statement,248.386919,chairman statement
2,Highlights,204.876537,highlights
3,Chief Executive’s Review,117.712061,chief executive review
4,Chief Executive’s review,104.321819,chief executive review


In [10]:
for index, row in train_df_with_score.iterrows():
    if index % 10 == 0:
        print('processing row number' , index)
    train_df_with_score.at[index, 'spanish'] = translate(row['section_name'], 'en', 'es')
    train_df_with_score.at[index, 'greek'] = translate(row['section_name'], 'en', 'el')

processing row number 0
processing row number 10
processing row number 20
processing row number 30
processing row number 40


In [11]:
train_df_with_score['clean_spanish'] = train_df_with_score.spanish.apply(lambda x: cleanup_sentence(x, 'es'))
train_df_with_score['clean_greek'] = train_df_with_score.greek.apply(lambda x: cleanup_sentence(x, 'el'))
train_df_with_score.head()

,section_name,score,clean_english,spanish,greek,clean_spanish,clean_greek
0,Chairman’s Statement,258.323541,chairman statement,Declaración del presidente,Δήλωση Προέδρου,declaración presidente,δήλωση προέδρου
1,Chairman’s statement,248.386919,chairman statement,declaración del presidente,Δήλωση του προέδρου,declaración presidente,δήλωση προέδρου
2,Highlights,204.876537,highlights,Reflejos,Καλύτερες στιγμές,reflejos,καλύτερες στιγμές
3,Chief Executive’s Review,117.712061,chief executive review,Revisión del director ejecutivo,Επιθεώρηση Διευθύνοντος Συμβούλου,revisión director ejecutivo,επιθεώρηση διευθύνοντος συμβούλου
4,Chief Executive’s review,104.321819,chief executive review,Revisión del director ejecutivo,Αναθεώρηση του Διευθύνοντος Συμβούλου,revisión director ejecutivo,αναθεώρηση διευθύνοντος συμβούλου


In [15]:
train_df_with_score.shape

(50, 7)

In [12]:
train_df_with_score.to_csv('multi_lingual_cleaned_toc_sections_original.csv', index=False)

In [13]:
import pickle
pickle.dump(train_df_with_score, open('multi_lingual_cleaned_toc_sections_original.pkl', 'wb'))